### Hito2 para Rendimiento Escolar
*"Se generará un análisis descriptivo considerando el tipo de variables (desde el punto de vista estadístico así como computacional). Esta sección debe considerar medidas univariadas/ frecuencias, datos perdidos y gráficos distributivos sobre las variables a analizar. A partir de ésta se debe clarificar la estrategia de preprocesamiento (datos perdidos, recodificaciones)."*

### Inspección Visual

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

import statsmodels.formula as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, classification_report 

from calidad_datos import calidad_datos
from graficos import cross_plot

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn') # Gráficos estilo seaborn
plt.rcParams["figure.figsize"] = (5, 3) # Tamaño gráficos (8, 6)
plt.rcParams["figure.dpi"] = 100 # resolución gráficos 70

In [63]:
df = pd.read_csv('students.csv', sep='|',index_col=0)
age=df.age.str.replace('"',"").replace(['nulidade','sem validade'],[np.nan,np.nan]).astype('float')
goout=df.goout.str.replace('"',"").replace(['nulidade','sem validade', 'zero'],[np.nan,np.nan,np.nan]).astype('float')
health=df.health.str.replace('"',"").replace(['sem validade', 'zero'],[np.nan,np.nan]).astype('float')

G1=df.G1.replace((['nulidade', 'zero'],[np.nan,np.nan])).astype('float')
G2=df.G1.value_counts()

G2



10          50
8           41
11          38
7           36
12          35
13          33
9           31
14          30
15          24
6           23
16          22
18           8
17           8
5            7
19           3
nulidade     3
zero         1
4            1
3            1
Name: G1, dtype: int64

### Proceso de preprocesamiento y recodificación de las variables